Find out what factors influence kwh of energy used per state<br>
Account for weather deviation from room temp<br>
Account for percent of energy that comes from electricity

# Data Collection / Cleaning

In [124]:
import pandas as pd
import numpy as np
from pandasql import sqldf
pysql = lambda q: sqldf(q, globals())

In [125]:
data = pd.DataFrame() # This will become the main dataset as I add columns from other datasets

## Energy per Capita

In [126]:
e_per_cap = pd.read_csv("energy_per_capita.csv")
# Source: U.S. Energy Information Administration (https://www.eia.gov/state/rankings/)
e_per_cap.head()

,Rank,State,"Total Energy Consumed per Capita, million Btu",Note: Rankings are based on the full source data values.
0,1,WY,932,NaN
1,2,LA,922,NaN
2,3,ND,875,NaN
3,4,AK,839,NaN
4,5,IA,517,NaN


In [127]:
e_per_cap = e_per_cap.sort_values(by=['State'])
e_per_cap.head()

,Rank,State,"Total Energy Consumed per Capita, million Btu",Note: Rankings are based on the full source data values.
3,4,AK,839,NaN
12,13,AL,392,NaN
16,17,AR,362,NaN
45,46,AZ,213,NaN
49,50,CA,198,NaN


In [128]:
data['state'] = e_per_cap['State']
data['energy_consum_per_cap_mil_btu'] = e_per_cap['Total Energy Consumed per Capita, million Btu']
data = data.reset_index(drop = True)
data.head()

,state,energy_consum_per_cap_mil_btu
0,AK,839
1,AL,392
2,AR,362
3,AZ,213
4,CA,198


## Population Density

In [129]:
popdens = pd.read_csv("apportionment.csv")
# Source: U.S. Census Bureau (https://www.census.gov/data/tables/time-series/dec/density-data-text.html)

In [130]:
popdens = popdens[popdens['Year'] == 2020].reset_index()
popdens.head()

,index,Name,Geography Type,Year,Resident Population,Percent Change in Resident Population,Resident Population Density,Resident Population Density Rank,Number of Representatives,Change in Number of Representatives,Average Apportionment Population Per Representative
0,627,Alabama,State,2020,"5,024,279",5.1,99.2,29.0,7.0,0.0,"718,579"
1,628,Alaska,State,2020,"733,391",3.3,1.3,52.0,1.0,0.0,"736,081"
2,629,Arizona,State,2020,"7,151,502",11.9,62.9,35.0,9.0,0.0,"795,436"
3,630,Arkansas,State,2020,"3,011,524",3.3,57.9,36.0,4.0,0.0,"753,439"
4,631,California,State,2020,"39,538,223",6.1,253.7,13.0,52.0,-1.0,"761,091"


In [131]:
popdens = popdens.drop(39).reset_index(drop=True)
popdens = popdens[popdens['Geography Type'] == 'State']
popdens = popdens.rename(columns={'Resident Population Density':'pop_density'})

In [132]:
popdens.head()

,index,Name,Geography Type,Year,Resident Population,Percent Change in Resident Population,pop_density,Resident Population Density Rank,Number of Representatives,Change in Number of Representatives,Average Apportionment Population Per Representative
0,627,Alabama,State,2020,"5,024,279",5.1,99.2,29.0,7.0,0.0,"718,579"
1,628,Alaska,State,2020,"733,391",3.3,1.3,52.0,1.0,0.0,"736,081"
2,629,Arizona,State,2020,"7,151,502",11.9,62.9,35.0,9.0,0.0,"795,436"
3,630,Arkansas,State,2020,"3,011,524",3.3,57.9,36.0,4.0,0.0,"753,439"
4,631,California,State,2020,"39,538,223",6.1,253.7,13.0,52.0,-1.0,"761,091"


In [133]:
abbrev = pd.read_csv("state_abbrev.csv") # this is to line up state names with correct
# abbreviations to merge with energy per capita data
# Source: World Population Review (https://worldpopulationreview.com/states/state-abbreviations)
abbrev.head()

,State,Abbrev,Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA


In [134]:
popdens['state'] = abbrev['Code']
popdens.head()

,index,Name,Geography Type,Year,Resident Population,Percent Change in Resident Population,pop_density,Resident Population Density Rank,Number of Representatives,Change in Number of Representatives,Average Apportionment Population Per Representative,state
0,627,Alabama,State,2020,"5,024,279",5.1,99.2,29.0,7.0,0.0,"718,579",AL
1,628,Alaska,State,2020,"733,391",3.3,1.3,52.0,1.0,0.0,"736,081",AK
2,629,Arizona,State,2020,"7,151,502",11.9,62.9,35.0,9.0,0.0,"795,436",AZ
3,630,Arkansas,State,2020,"3,011,524",3.3,57.9,36.0,4.0,0.0,"753,439",AR
4,631,California,State,2020,"39,538,223",6.1,253.7,13.0,52.0,-1.0,"761,091",CA


In [135]:
q = """
select state, name, energy_consum_per_cap_mil_btu, pop_density from
(select * from data left join popdens on popdens.state = data.state);
"""

In [136]:
data = pysql(q)
data.head()

,state,Name,energy_consum_per_cap_mil_btu,pop_density
0,AK,Alaska,839,1.3
1,AL,Alabama,392,99.2
2,AR,Arkansas,362,57.9
3,AZ,Arizona,213,62.9
4,CA,California,198,253.7


## Political Views

In [137]:
pol = pd.read_csv("1976-2020-president.csv")

In [138]:
pol.head()

,year,state,state_po,state_fips,state_cen,state_ic,office,candidate,party_detailed,writein,candidatevotes,totalvotes,version,notes,party_simplified
0,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"CARTER, JIMMY",DEMOCRAT,False,659170,1182850,20210113,NaN,DEMOCRAT
1,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"FORD, GERALD",REPUBLICAN,False,504070,1182850,20210113,NaN,REPUBLICAN
2,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"MADDOX, LESTER",AMERICAN INDEPENDENT PARTY,False,9198,1182850,20210113,NaN,OTHER
3,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"BUBAR, BENJAMIN """"BEN""""",PROHIBITION,False,6669,1182850,20210113,NaN,OTHER
4,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"HALL, GUS",COMMUNIST PARTY USE,False,1954,1182850,20210113,NaN,OTHER


In [139]:
q = """
select state, state_po, party_detailed, candidatevotes, totalvotes from
pol where year = 2020 and party_detailed in ("DEMOCRAT", "REPUBLICAN");
"""
pol = pysql(q)

In [140]:
pol.head()

,state,state_po,party_detailed,candidatevotes,totalvotes
0,ALABAMA,AL,DEMOCRAT,849624,2323282
1,ALABAMA,AL,REPUBLICAN,1441170,2323282
2,ALASKA,AK,DEMOCRAT,153778,359530
3,ALASKA,AK,REPUBLICAN,189951,359530
4,ARIZONA,AZ,DEMOCRAT,1672143,3387326


In [141]:
c = np.zeros(102)
dem = pol[pol['party_detailed'] == 'DEMOCRAT']
rep = pol[pol['party_detailed'] == 'REPUBLICAN']
democrat = pd.DataFrame({'state': pol['state_po'], 'dem': dem['candidatevotes']/dem['totalvotes']}).dropna()
republican = pd.DataFrame({'state': pol['state_po'], 'rep': rep['candidatevotes']/rep['totalvotes']}).dropna()

In [146]:
q = """
select data.*, democrat.dem, republican.rep from democrat join republican on democrat.state = republican.state
join data on republican.state = data.state;
"""
data = pysql(q)

In [148]:
data.head()

,state,dem,rep,Name,energy_consum_per_cap_mil_btu,pop_density,dem,rep
0,AL,0.365700,0.620316,Alabama,392,99.2,0.365700,0.620316
1,AK,0.427720,0.528331,Alaska,839,1.3,0.427720,0.528331
2,AZ,0.493647,0.490560,Arizona,213,62.9,0.493647,0.490560
3,AR,0.347751,0.623957,Arkansas,362,57.9,0.347751,0.623957
4,CA,0.634839,0.343207,California,198,253.7,0.634839,0.343207


## Weather

Source: https://www.ncei.noaa.gov/access/us-climate-normals/
For weather, I couldn't find any good data by state, so I downloaded csv files on weather for stations in the most populous city in each state. For California and Florida, I am going to use weighted averages based on population for Los Angeles/San Francisco and Jacksonville/Miami respectively since they are very far apart and populous. I got city populations from https://en.wikipedia.org/wiki/List_of_largest_cities_of_U.S._states_and_territories_by_population

In [197]:
sf = pd.read_csv("weather_data/sanfrancisco.csv")
la = pd.read_csv("weather_data/losangeles.csv")
mi = pd.read_csv("weather_data/miami.csv")
jv = pd.read_csv("weather_data/jacksonville.csv")

In [198]:
sfpop = 875000
lapop = 3967000
mipop = 454000
jvpop = 890000

camax = (sf["MLY-TMAX-NORMAL"]*(sfpop/(sfpop+lapop)) + la["MLY-TMAX-NORMAL"]*(lapop/(sfpop+lapop))).round(1)
camin = (sf["MLY-TMIN-NORMAL"]*(sfpop/(sfpop+lapop)) + la["MLY-TMIN-NORMAL"]*(lapop/(sfpop+lapop))).round(1)
CA = pd.DataFrame({"MLY-TMAX-NORMAL": camax, "MLY-TMIN-NORMAL": camin})

flmax = (mi["MLY-TMAX-NORMAL"]*(mipop/(mipop+jvpop)) + jv["MLY-TMAX-NORMAL"]*(jvpop/(mipop+jvpop))).round(1)
flmin = (mi["MLY-TMIN-NORMAL"]*(mipop/(mipop+jvpop)) + jv["MLY-TMIN-NORMAL"]*(jvpop/(mipop+jvpop))).round(1)
FL = pd.DataFrame({"MLY-TMAX-NORMAL": flmax, "MLY-TMIN-NORMAL": flmin})

Now we have dataframes for California and Florida. Next, I'll create dataframes for the rest of the states:

In [199]:
states = list(data['Name'])
states.remove('California')
states.remove('Florida')

In [200]:
d = {}
for s in states:
    d[s] = pd.read_csv(f'weather_data/{s}.csv')
d['California'] = CA
d['Florida'] = FL

In [201]:
def room_dev(df):
    """
    Returns metric for deviation from room temperature.
    
    Takes absolute difference between 70 degrees and the monthly high averages, then sums the months.
    The same is done for the monthly low averages and 65 degrees and these sums are added together.
    """
    high = abs(df['MLY-TMAX-NORMAL']-70)
    low = abs(df['MLY-TMIN-NORMAL']-65)
    return np.sum(high + low).round(1)

In [202]:
for k in d.keys():
    d[k] = room_dev(d[k])

In [203]:
weather = pd.DataFrame({'state': list(d.keys()), 'weather': list(d.values())})

In [204]:
q = """
select data.*, weather.weather from data join weather on weather.state = data.Name;
"""
data = pysql(q)

In [205]:
data.head()

,state,dem,rep,Name,energy_consum_per_cap_mil_btu,pop_density,weather,weather
0,AL,0.365700,0.620316,Alabama,392,99.2,340.7,340.7
1,AK,0.427720,0.528331,Alaska,839,1.3,719.1,719.1
2,AZ,0.493647,0.490560,Arizona,213,62.9,347.5,347.5
3,AR,0.347751,0.623957,Arkansas,362,57.9,342.9,342.9
4,CA,0.634839,0.343207,California,198,253.7,155.9,155.9
